In [ ]:
# Fix for NotImplementedError for notebook in Python3.8
# https://stackoverflow.com/questions/58422817/jupyter-notebook-with-python-3-8-notimplementederror

In [5]:
import networkx as nx
from vertex_cover import vertex_cover_kernelization